<a href="https://colab.research.google.com/github/lavratti/aulas_interpretadores/blob/main/Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install chatterbot

from chatterbot import ChatBot
from chatterbot.trainers import ListTrainer
import nltk
nltk.download('punkt')
import re
import requests
from bs4 import BeautifulSoup

url_list = [
            "https://pt.wikipedia.org/wiki/Ucr%C3%A2nia",
            "https://pt.wikipedia.org/wiki/R%C3%BAssia",
            "https://pt.wikipedia.org/wiki/Invas%C3%A3o_da_Ucr%C3%A2nia_pela_R%C3%BAssia_em_2022",
            "https://pt.wikipedia.org/wiki/Guerra_em_Donbas",
            "https://pt.wikipedia.org/wiki/Anexa%C3%A7%C3%A3o_da_Crimeia_%C3%A0_Federa%C3%A7%C3%A3o_Russa",
            "https://g1.globo.com/mundo/ucrania-russia/",
            "https://www.cnnbrasil.com.br/tudo-sobre/guerra-na-ucrania/",
            "https://www1.folha.uol.com.br/mundo/guerra-da-ucrania/",
            "https://noticias.r7.com/internacional/russia-x-ucrania",
            "https://aovivo.folha.uol.com.br/mundo/2022/03/31/6113-acompanhe-as-principais-noticias-sobre-a-guerra-da-ucrania.shtml",
            "https://brasilescola.uol.com.br/geografia/por-que-a-russia-invadiu-a-ucrania-em-2022.htm",
            "https://www.dw.com/pt-002/guerra-na-ucr%C3%A2nia/t-60927282",
            ]


all_senteces = []
for url in url_list:
  page = requests.get(url)
  page = page.text
  soup = BeautifulSoup(page, features="lxml")
  for script in soup(["script", "style"]):
      script.decompose()
  raw_text  = soup.get_text()

  cleaned = re.sub(r"(?is)<(script|style).*?>.*?(</\1>)", "", raw_text.strip())
  cleaned = re.sub(r"(?s)<!--(.*?)-->[\n]?", "", cleaned)
  cleaned = re.sub(r"(?s)<.*?>", " ", cleaned)
  cleaned = re.sub(r"&nbsp;", " ", cleaned)
  cleaned = re.sub(r"  ", " ", cleaned)
  cleaned = re.sub("\n", " ", cleaned)
  cleaned = re.sub("'", " ", cleaned)
  cleaned = re.sub("\"", " ", cleaned)
  cleaned = cleaned.strip()
  sent_tokens = nltk.sent_tokenize(cleaned)
  for token in sent_tokens:
    all_senteces.append(token)



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
bot = ChatBot(
    'ReporterBot',
    storage_adapter='chatterbot.storage.SQLStorageAdapter',
    database_uri='sqlite:///database.sqlite3'
    )
    
chat = ListTrainer(bot)
chat.train(all_senteces)

List Trainer: [####################] 100%


In [ ]:
while True:
  response = bot.get_response(input("Pergunta: "))
  print("[Confiança na resposta: {:}%]\nResposta:".format(100*float(response.confidence)), response)

